In [1]:
import random
import numpy as np
from scipy.stats import multivariate_normal, norm
from scipy.optimize import minimize
import pandas as pd

In [2]:
###### FLEMING'S FUNCTION MODIFIED
def compute_meta_conf_serialdependence(xp, a, sigma_act, sigma_conf, rho):
    
    dhat = np.array([-1, 1])
    mu_x_xp_dhat = np.zeros((2, len(xp)))
    var_x_xp_dhat = np.zeros(len(xp))
    rho_vec = np.full(len(xp), rho)
    sigA_vec = np.full(len(xp), sigma_act)
    sigP_vec = np.full(len(xp), sigma_conf)
    
    Tol = 10e-4

    for dhati in range(2):
        dhat_vec = np.full(len(xp), dhat[dhati])
        
        mu_x_xp_dhat[dhati, :] = dhat_vec + (sigA_vec / sigP_vec) * rho_vec * (xp - dhat_vec)
        var_x_xp_dhat = (1 - rho_vec**2) * sigA_vec**2
        
        if a == 1:
            p_a_dhat_xp = 1 - norm.cdf(0, mu_x_xp_dhat[dhati, :], np.sqrt(var_x_xp_dhat))
        else:
            p_a_dhat_xp = norm.cdf(0, mu_x_xp_dhat[dhati, :], np.sqrt(var_x_xp_dhat))
        
        lik_d = norm.pdf(xp, dhat_vec, sigP_vec)
        
        if dhati == 0:
            p_a_dhat_xp_full = p_a_dhat_xp
            lik_d_full = lik_d
        else:
            p_a_dhat_xp_full = np.vstack((p_a_dhat_xp_full, p_a_dhat_xp))
            lik_d_full = np.vstack((lik_d_full, lik_d))
    
    # manage probability
    p_a_dhat_xp_full = np.clip(p_a_dhat_xp_full, Tol, None)
    lik_d_full = np.clip(lik_d_full, Tol, None)
    
    lik_d_full = lik_d_full / np.sum(lik_d_full, axis=0, keepdims=True)
    p_dhat_xp_a = p_a_dhat_xp_full * lik_d_full
    p_dhat_xp_a = p_dhat_xp_a / np.sum(p_dhat_xp_a, axis=0, keepdims=True)
    
    # Conf = p(a=d)
    if a == 1:
        conf = p_dhat_xp_a[1, :]
    else:
        conf = p_dhat_xp_a[0, :]
    
    return conf

In [3]:
# Función que devuelve una confianza con manipulación de dependencia serial y sin ella
def model_run(d, a, rho, sigmaConf, theta, alpha_sd, beta_sd):
    sigmaAct = 1
    bigSigma = np.array([[sigmaAct**2, rho * sigmaAct * sigmaConf], [rho * sigmaAct * sigmaConf, sigmaConf**2]])

    N = len(d)
    xa = np.empty(N)
    xp = np.empty(N)
    secondOrder_mean_cor_serialDependence =  np.full( N, 111.0)
    first_trial = True 

    for i in range(N):
        current_theta = theta
        r = multivariate_normal.rvs(mean=[d[i] * current_theta, d[i] * current_theta], cov=bigSigma)
        
        xa[i] = r[0]
        xp[i] = r[1]
        
        flip_a = a[i]
        
        if first_trial == False:
            p_serial_dependence = np.random.beta(alpha_sd, beta_sd, 1)[0]
            secondOrder_mean_cor_serialDependence[i] = (
                p_serial_dependence * secondOrder_mean_cor_serialDependence[i-1] + 
                (1-p_serial_dependence) * compute_meta_conf_serialdependence(np.array([xp[i]]), flip_a, sigmaAct, sigmaConf, rho)[0]
            )
        else:
            secondOrder_mean_cor_serialDependence[i] = compute_meta_conf_serialdependence(np.array([xp[i]]), flip_a, sigmaAct, sigmaConf, rho)[0]
    
        first_trial = False

    # drop first value
    secondOrder_mean_cor_serialDependence_adj = secondOrder_mean_cor_serialDependence[1:]

    return secondOrder_mean_cor_serialDependence_adj

# Cargar datos y correr la optimización de los parámetros
df = pd.read_csv('data_Mazancieux_2018.csv')
#suj = [1,2,3]
#df_vp = df[(df['Task'] == 'VP') & (df['Subj_idx'].isin(suj))].copy()
df_vp = df[df['Task'] == 'EM'].copy()
df_vp.loc[:, 'Stimulus_transformed'] = df_vp['Stimulus'].replace({1: -1, 2: 1})
df_vp.loc[:, 'Response_transformed'] = df_vp['Response'].replace({1: 0, 2: 1})
df_vp['Confidence_0to1'] = df_vp['Confidence'] / 10

n_participants = len(df_vp['Subj_idx'].unique())

def negative_log_likelihood(params):
    rho = params[0]
    sigmaConf = params[1]
    theta = params[2]
    alpha_sd = params[3]
    beta_sd = params[4]

    all_secondOrder_mean_cor_serialDependence = []

    for n_p in range(1, n_participants + 1):
        df_vp_participant = df_vp[df_vp['Subj_idx'] == n_p]
        d = df_vp_participant['Stimulus_transformed'].values
        a = df_vp_participant['Response_transformed'].values
        
        secondOrder_mean_cor = model_run(d, a, rho, sigmaConf, theta, alpha_sd, beta_sd)

        all_secondOrder_mean_cor_serialDependence.extend(secondOrder_mean_cor)

    all_secondOrder_mean_cor_serialDependence = np.array(all_secondOrder_mean_cor_serialDependence)

    #df_vp_cleaned = df_vp.groupby('Subj_idx', group_keys=False).apply(lambda x: x.iloc[1:]).reset_index(drop=True)
    grouped = df_vp.groupby('Subj_idx')
    df_vp_cleaned = pd.concat([group.iloc[1:] for _, group in grouped]).reset_index(drop=True)
    conf_data = df_vp_cleaned['Confidence_0to1'].values
   
    log_likelihood = np.sum(norm.logpdf(conf_data, all_secondOrder_mean_cor_serialDependence))
    
    print(f"Parameters: rho={rho}, sigmaConf={sigmaConf}, theta={theta}, alpha: {alpha_sd}, beta {beta_sd}, Log-Likelihood: {-log_likelihood}")  # Añadí esta línea para imprimir los parámetros y el log-likelihood

    return -log_likelihood

# Inicializar valores iniciales para los parámetros
initial_guess = [0.5, 1.0, 0.5, 3, 7]  
bounds = [(0.0001, 0.9999), (0.1, 2), (0.1, 0.9), (1, 20), (1, 20)]


result = minimize(negative_log_likelihood, initial_guess, bounds=bounds, method='Powell') # 'L-BFGS-B'
optimal_params = result.x
optimal_rho = optimal_params[0]
optimal_sigmaConf = optimal_params[1]
optimal_theta = optimal_params[2]
optimal_alpha_sd = optimal_params[3]
optimal_beta_sd = optimal_params[4]

print("Optimal rho:", optimal_rho)
print("Optimal sigmaConf:", optimal_sigmaConf)
print("Optimal theta:", optimal_theta)
print("Optimal alpha sd:", optimal_alpha_sd)
print("Optimal beta sd:", optimal_beta_sd)


Parameters: rho=0.5, sigmaConf=1.0, theta=0.5, alpha: 3.0, beta 7.0, Log-Likelihood: 7030.584443254078
Parameters: rho=0.3819896180478551, sigmaConf=1.0, theta=0.5, alpha: 3.0, beta 7.0, Log-Likelihood: 7041.926466332486
Parameters: rho=0.6180103819521449, sigmaConf=1.0, theta=0.5, alpha: 3.0, beta 7.0, Log-Likelihood: 7024.014495078093
Parameters: rho=0.7638792360957102, sigmaConf=1.0, theta=0.5, alpha: 3.0, beta 7.0, Log-Likelihood: 7007.738809230555
Parameters: rho=0.8540311458564346, sigmaConf=1.0, theta=0.5, alpha: 3.0, beta 7.0, Log-Likelihood: 7013.770414737164
Parameters: rho=0.7647184904578577, sigmaConf=1.0, theta=0.5, alpha: 3.0, beta 7.0, Log-Likelihood: 7012.84549381104
Parameters: rho=0.7081622917128692, sigmaConf=1.0, theta=0.5, alpha: 3.0, beta 7.0, Log-Likelihood: 7025.420367093838
Parameters: rho=0.7425972570907524, sigmaConf=1.0, theta=0.5, alpha: 3.0, beta 7.0, Log-Likelihood: 7005.736439902369
Parameters: rho=0.7530645329468485, sigmaConf=1.0, theta=0.5, alpha: 3.0

In [4]:
'''
Optimal rho: 0.944183055617159
Optimal sigmaConf: 0.9959047251414038
Optimal theta: 0.2167184270002524
Optimal alpha sd: 13.217373954433665
Optimal beta sd: 5.4852915724960045
'''

'\nOptimal rho: 0.944183055617159\nOptimal sigmaConf: 0.9959047251414038\nOptimal theta: 0.2167184270002524\nOptimal alpha sd: 13.217373954433665\nOptimal beta sd: 5.4852915724960045\n'

In [5]:
import csv

# Guardar en un archivo CSV
with open('EMtask_optimal_parameters_model_4_WITH_SerialDependence.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Parameter', 'Value'])
    writer.writerow(['rho', optimal_rho])
    writer.writerow(['sigmaConf', optimal_sigmaConf])
    writer.writerow(['theta', optimal_theta])
    writer.writerow(['alpha_sd', optimal_alpha_sd])
    writer.writerow(['beta_sd', optimal_beta_sd])